https://keras.io/examples/imdb_lstm/

In [ ]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, concatenate
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

In [ ]:


# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)




Loading data...
17465344/17464789 [==============================] - 1s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [ ]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(MaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 50)           250000    
_________________________________________________________________
dropout (Dropout)            (None, 400, 50)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 398, 250)          37750     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 199, 250)          0         
_________________________________________________________________
dense (Dense)                (None, 199, 250)          62750     
_________________________________________________________________
dropout_1 (Dropout)          (None, 199, 250)          0         
_________________________________________________________________
activation (Activation)      (None, 199, 

In [ ]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
print('Train...')

checkpoint = EarlyStopping(monitor='val_acc', mode='max')
callbacks_lst = [checkpoint]

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test),
          callbacks=callbacks_lst)

history_dict = history.history
history_dict.keys()

score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15
782/782 [==============================] - 139s 178ms/step - loss: 0.6776 - accuracy: 0.5434 - val_loss: 0.6747 - val_accuracy: 0.5478
Epoch 2/15
782/782 [==============================] - 140s 179ms/step - loss: 0.6714 - accuracy: 0.5572 - val_loss: 0.6755 - val_accuracy: 0.5529
Epoch 3/15
782/782 [==============================] - 140s 179ms/step - loss: 0.6698 - accuracy: 0.5560 - val_loss: 0.6726 - val_accuracy: 0.5557
Epoch 4/15
782/782 [==============================] - 141s 180ms/step - loss: 0.6683 - accuracy: 0.5589 - val_loss: 0.6719 - val_accuracy: 0.5524
Epoch 5/15
782/782 [==============================] - 139s 178ms/step - loss: 0.6671 - accuracy: 0.5606 - val_loss: 0.6722 - val_accuracy: 0.5522
Epoch 6/15
782/782 [==============================] - 141s 180ms/step - loss: 0.6659 - accuracy: 0.5647 - val_loss: 0.6727 - val_accuracy: 0.5565
Epoch 7/15
782/782 [==============================] - 137s 175ms/step - loss: 0.6652 - accuracy: 0.5646 - val_loss:

KeyboardInterrupt: ignored

As you can see, the output of the Conv1D and Maxpool1D is 3D tensor, including the temporal info (pooled over 4). We need to summarize this dimension.

We have few options:



- Flatten

- GlobalMaxPool1D: which takes the max over the temporal and feature dimensions https://stackoverflow.com/questions/43728235/what-is-the-difference-between-keras-maxpooling1d-and-globalmaxpooling1d-functi/43730861

- LSTM



# Maxpool1D + Flatten

In [ ]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(MaxPooling1D())
model.add(Flatten())
# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

Build model...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 400, 50)           250000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 199, 250)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 49750)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 250)               12437750  
_________________________________________________________________
dropout_5 (Dropout)          (None, 250

In [ ]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
print('Train...')

checkpoint = EarlyStopping(monitor='val_acc', mode='max')
callbacks_lst = [checkpoint]

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test),
          callbacks=callbacks_lst)

history_dict = history.history
history_dict.keys()

score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15
782/782 [==============================] - 151s 193ms/step - loss: 0.3687 - accuracy: 0.8245 - val_loss: 0.2777 - val_accuracy: 0.8843
Epoch 2/15
782/782 [==============================] - 153s 195ms/step - loss: 0.2156 - accuracy: 0.9151 - val_loss: 0.3085 - val_accuracy: 0.8698
Epoch 3/15
782/782 [==============================] - 154s 197ms/step - loss: 0.1596 - accuracy: 0.9399 - val_loss: 0.3412 - val_accuracy: 0.8722
Epoch 4/15
 39/782 [>.............................] - ETA: 1:52 - loss: 0.0932 - accuracy: 0.9712

KeyboardInterrupt: ignored

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

NameError: ignored

In [ ]:

plt.clf()   # clear figure
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

NameError: ignored

<Figure size 432x288 with 0 Axes>

# GlobalMaxPool1D

In [ ]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

In [ ]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
print('Train...')

checkpoint = EarlyStopping(monitor='val_acc', mode='max')
callbacks_lst = [checkpoint]

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test),
          callbacks=callbacks_lst)

history_dict = history.history
history_dict.keys()

score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
history_dict.keys()

In [ ]:

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:

plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# CNN+LSTM

In [ ]:
# Embedding
max_features = 20000
maxlen = 100
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
#batch_size = 30
#epochs = 2

In [ ]:
print('Build model...')

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

Build model...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 96, 64)            41024     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 70)                37800     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 71        
_________________________________________________________________
activation_6 (Activation)    (None, 1) 

In [ ]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
print('Train...')

checkpoint = EarlyStopping(monitor='val_acc', mode='max')
callbacks_lst = [checkpoint]

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test),
          callbacks=callbacks_lst)

history_dict = history.history
history_dict.keys()

score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15
782/782 [==============================] - 149s 190ms/step - loss: 0.3543 - accuracy: 0.8360 - val_loss: 0.2666 - val_accuracy: 0.8897
Epoch 2/15
782/782 [==============================] - 150s 192ms/step - loss: 0.2111 - accuracy: 0.9191 - val_loss: 0.3096 - val_accuracy: 0.8759
Epoch 3/15
782/782 [==============================] - 152s 195ms/step - loss: 0.1546 - accuracy: 0.9427 - val_loss: 0.3081 - val_accuracy: 0.8907
Epoch 4/15
782/782 [==============================] - 150s 192ms/step - loss: 0.1049 - accuracy: 0.9641 - val_loss: 0.3047 - val_accuracy: 0.8830
Epoch 5/15
782/782 [==============================] - 151s 193ms/step - loss: 0.0732 - accuracy: 0.9757 - val_loss: 0.3636 - val_accuracy: 0.8801
Epoch 6/15
782/782 [==============================] - 152s 194ms/step - loss: 0.0515 - accuracy: 0.9827 - val_loss: 0.3980 - val_accuracy: 0.8824
Epoch 7/15
782/782 [==============================] - 152s 195ms/step - loss: 0.0409 - accuracy: 0.9859 - val_loss:

In [ ]:

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

KeyError: ignored

In [ ]:

plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Multi-channel n-gram CNN:

The above CNN has only 1 field of view: filter over every 3 words = trigrams

What if we want to have features over different spans?:

- 4-gram
- 6-gram
- 8-gram

We can use multi-input model.

For that, we will use the functional API of Keras.

Notice how the 3 input streams are merged through concatenate layer:

In [ ]:
inputs1 = Input(shape=(maxlen,))
embedding1 = Embedding(max_features, embedding_size)(inputs1)
conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPooling1D(pool_size=2)(drop1)
flat1 = Flatten()(pool1)
# channel 2
inputs2 = Input(shape=(maxlen,))
embedding2 = Embedding(max_features, embedding_size, input_length=maxlen)(inputs2)
conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
drop2 = Dropout(0.5)(conv2)
pool2 = MaxPooling1D(pool_size=2)(drop2)
flat2 = Flatten()(pool2)
# channel 3
inputs3 = Input(shape=(maxlen,))
embedding3 = Embedding(max_features, embedding_size, input_length=maxlen)(inputs3)
conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
drop3 = Dropout(0.5)(conv3)
pool3 = MaxPooling1D(pool_size=2)(drop3)
flat3 = Flatten()(pool3)
# merge
merged = concatenate([flat1, flat2, flat3])
# interpretation
dense1 = Dense(10, activation='relu')(merged)
outputs = Dense(1, activation='sigmoid')(dense1)
model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
# compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# summarize
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True, to_file='multichannel.png')

Notice how the input is now a list of 3: repeated versions of the input. This is because Keras is a static graph framework.

In [ ]:
print('Train...')

checkpoint = EarlyStopping(monitor='val_acc', mode='max')
callbacks_lst = [checkpoint]

history = model.fit([x_train, x_train, x_train], y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=([x_test, x_test, x_test], y_test),
          callbacks=callbacks_lst)

history_dict = history.history
history_dict.keys()

score, acc = model.evaluate([x_test, x_test, x_test], y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)